In [66]:
pwd

'/home/wsuser/work'

In [67]:
import warnings
warnings.filterwarnings("ignore")

In [68]:
!pip install imutils

Image preprocessing


Importing the necessary libraries

In [69]:
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from skimage import feature
from imutils import paths
import os
import pickle
from ibm_watson_machine_learning import APIClient
import json

Functions to load and quantify images

In [70]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='EnmC5rosoHPqN3zlyKo5fYxmwNOFaReRLlgM8wcMS-bk',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'parkinson-donotdelete-pr-eflnpymujxd7hu'
object_key = 'dataset1.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [71]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [72]:
pwd

'/home/wsuser/work'

In [73]:
def quantify_image(image):
    features = feature.hog(image, 
                           orientations=9, 
                           pixels_per_cell=(5,5), 
                           cells_per_block=(2,2), 
                           transform_sqrt=True, 
                           block_norm="L1")
    return features

In [74]:
def load_split(path):
    path_images = list(paths.list_images(path))
    data=[]
    labels=[]

    for path_image in path_images:
        label = path_image.split(os.path.sep)[-2]
        image = cv2.imread(path_image)
        image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image, (200,200))
        image = cv2.threshold(image,0,225,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

        features = quantify_image(image)
        data.append(features)
        labels.append(label)

    return (np.array(data), np.array(labels))

Using spiral images

Defining the path for training data and testing data

In [75]:
path_training_data = r"/home/wsuser/work/dataset/spiral/training"
path_testing_data = r"/home/wsuser/work/dataset/spiral/testing"

Loading the training and testing data

In [76]:
(x_train, y_train) = load_split(path_training_data)
(x_test, y_test) = load_split(path_testing_data)

Label Encoding

In [77]:
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)
print(x_train.shape, y_train.shape)
# 0:healthy,1:Parkinson

(144, 54756) (144,)


Building the model

In [78]:
model = RandomForestClassifier(n_estimators=100)
model.fit(x_train, y_train)

RandomForestClassifier()

Testing the model

In [79]:
testingPaths = list(paths.list_images(path_testing_data))
idxs = np.arange(0, len(testingPaths))
idxs = np.random.choice(idxs, size=(25,), replace=False) 
images = []

In [80]:
for i in idxs:
    # loading the testing image, clone it, and resize it 
    image = cv2.imread(testingPaths[i]) 
    output = image. copy() 
    output = cv2. resize(output, (128, 128))
    
    # pre-processing the image 
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
    image = cv2.resize(image, (200, 200))
    image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU) [1]
    
    # quantify the image and make predictions based on the extracted 
    # features using the last trained Random Forest 
    features = quantify_image(image) 
    preds = model.predict([features])

    label = label_encoder.inverse_transform(preds)[0]

    # draw the colored class label on the output image and add it to the set of output images 
    if label == "healthy":
        color = (0, 255, 0)  
    else:
        color = (0, 0, 255) 
    cv2.putText(output, label, (3, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2) 
    images.append(output)

Evaluating the model

In [81]:
predictions = model.predict(x_test)    # predictions on the testing data 

cm = confusion_matrix(y_test, predictions).flatten ()    # computing the confusion matrix
print(cm) 
(tn, fp, fn, tp) = cm 

accuracy = (tp + tn) / float(cm.sum())     # computing the accuracy
print(accuracy)

[26  4 10 20]
0.7666666666666667


Saving the model

In [82]:
pickle.dump(model,open('parkinson.pkl', 'wb'))

In [83]:
!tar -zcvf parkinsonsmodel.tgz parkinson.pkl

parkinson.pkl


In [84]:
ls

dataset/       parkinsons-detection-model_s.tgz  parkinson_w.pkl
parkinson.pkl  parkinsonsmodel.tgz


In [85]:
!pip install watson-machine-learning-client --upgrade

In [86]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
                    "url" : "https://us-south.ml.cloud.ibm.com",
                    "apikey" : "K36t7CdPrC_R4VWU_knCWisv6ePwSofTJbYqDvwtHL54"
                    }
client = APIClient(wml_credentials)

In [87]:
def guid_from_space_name(client,space_name):
    space = client.spaces.get_details()
    print(space)
    return (next(item for item in space['resources'] if item['entity']['name']==space_name)['metadata']['id'])

In [88]:
space_uid = guid_from_space_name(client,'ParkinsonsDiseaseDetection')
print('Space UID = ' + space_uid)

{'resources': [{'entity': {'compute': [{'crn': 'crn:v1:bluemix:public:pm-20:us-south:a/b84b413d6b9e4175b7855d4dd2ee33d9:9ba5da9b-52e4-4572-8e8d-fff28a6b8cc6::', 'guid': '9ba5da9b-52e4-4572-8e8d-fff28a6b8cc6', 'name': 'Watson Machine Learning-ki', 'type': 'machine_learning'}], 'description': '', 'name': 'ParkinsonsDiseaseDetection', 'scope': {'bss_account_id': 'b84b413d6b9e4175b7855d4dd2ee33d9'}, 'stage': {'production': False}, 'status': {'state': 'active'}, 'storage': {'properties': {'bucket_name': '5a89077f-3edd-4509-b6cb-d2ee2ab49e86', 'bucket_region': 'us-south', 'credentials': {'admin': {'access_key_id': '40e04ea2f6794d2b80c672bfeffc7d49', 'api_key': '3hGFaYijB8KkwMkcI_qtULkbD3eteT16p8zj8XHxLGtC', 'secret_access_key': '05736898bfeea8166d65dfe96f880c8b86568f8657a2de79', 'service_id': 'ServiceId-46cc9955-6c14-4654-8e28-6bddac6edf85'}, 'editor': {'access_key_id': 'b07c7bcf702144d4a375823ae0af96ec', 'api_key': 'D1Nyz0418QME4N9Uj5TM_8RKjnEdxDmViSP7OjpGgdrQ', 'resource_key_crn': 'crn:v1:

In [89]:
client.set.default_space(space_uid)

'SUCCESS'

In [90]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [91]:
software_spec_uid=client.software_specifications.get_uid_by_name("default_py3.6") 
software_spec_uid

'0062b8c9-8b7d-44a0-a9b9-46c416adcbd9'

model_details = client.repository.store_model(model='parkinsonsmodel.tgz',meta_props={
client.repository.ModelMetaNames.NAME: "parkinson", 
client.repository.ModelMetaNames.TYPE: "default_py3.6", 
client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid})
model_id = client.repository.get_model_uid(model_details)

Using wave images

Defining the path for wave training data and testing data

In [92]:
path_training_data = r"dataset/wave/training"
path_testing_data = r"dataset/wave/testing"

Loading the training and testing data

In [93]:
(x_train, y_train) = load_split(path_training_data)
(x_test, y_test) = load_split(path_testing_data)

Label Encoding

In [94]:
abel_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)
print(x_train.shape, y_train.shape)
# 0:healthy,1:Parkinson

(72, 54756) (72,)


Building the model

Training the model

In [95]:
model = RandomForestClassifier(n_estimators=100)
model.fit(x_train, y_train)

RandomForestClassifier()

Testing the model

In [96]:
testingPaths = list(paths.list_images(path_testing_data))
idxs = np.arange(0, len(testingPaths))
idxs = np.random.choice(idxs, size=(25,), replace=False) 
images = []

In [97]:
for i in idxs:
    # loading the testing image, clone it, and resize it 
    image = cv2.imread(testingPaths[i]) 
    output = image. copy() 
    output = cv2. resize(output, (128, 128))
    
    # pre-processing the image 
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
    image = cv2.resize(image, (200, 200))
    image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU) [1]
    
    # quantify the image and make predictions based on the extracted 
    # features using the last trained Random Forest 
    features = quantify_image(image) 
    preds = model.predict([features])

    label = label_encoder.inverse_transform(preds)[0]

    # draw the colored class label on the output image and add it to the set of output images 
    if label == "healthy":
        color = (0, 255, 0)  
    else:
        color = (0, 0, 255) 
    cv2.putText(output, label, (3, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2) 
    images.append(output)

Evaluating the model


In [113]:
predictions = model.predict(x_test)    # predictions on the testing data 

cm = confusion_matrix(y_test, predictions).flatten ()    # computing the confusion matrix
print(cm) 
(tn, fp, fn, tp) = cm 

accuracy = (tp + tn) / float(cm.sum())     # computing the accuracy
print(accuracy)

[20 10 12 18]
0.6333333333333333


Saving the model

In [114]:
pickle.dump(model,open('parkinson_w.pkl', 'wb'))

In [115]:
!tar -zcvf parkinsons-detection-model_s.tgz parkinson_w.pkl

parkinson_w.pkl


In [116]:
ls

dataset/       parkinsons-detection-model_s.tgz  parkinson_w.pkl
parkinson.pkl  parkinsonsmodel.tgz
